In [24]:

from selenium import webdriver
import requests
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

options=webdriver.ChromeOptions()
options.headless=True

driver=webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))


# Lien de la page web à scraper
url = "https://www.syride.com/fr/explorer/3764768529"
driver.get(url)


# Attendre que la page se charge complètement
driver.implicitly_wait(5)  # Attendre jusqu'à 10 secondes pour que les éléments se chargent
i=2
while i<4:
    
    # Récupérer toutes les classes lineDiv
    lineDivs = driver.find_elements(By.CLASS_NAME, "lineDiv")
    vols=[]
    # Pour chaque classe lineDiv, cliquer sur le bouton loupe et extraire le lien
    for lineDiv in lineDivs:
        loupe_button = lineDiv.find_element(By.CSS_SELECTOR, "td[onclick^='showFlight'] img")
        driver.implicitly_wait(1)
        driver.execute_script("arguments[0].click();", loupe_button)
    
        vol_url = driver.current_url
        print(vol_url)
        vols.append(vol_url)



    # Construire le sélecteur CSS avec la valeur de i
    css_selector = f'img.numeroPagesResultats[data-page="{i}"]'

    # Tenter de trouver l'élément avec le sélecteur CSS
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector))
    )

    # Exécuter un script JavaScript pour cliquer sur l'élément
    driver.execute_script("arguments[0].click();", element)
    
    print(f"Cliqué sur la page {i}")
    i += 1
    driver.implicitly_wait(5)

https://www.syride.com/fr/pilotes/Stephgoepp/2459041
https://www.syride.com/fr/pilotes/prabert/2458158
https://www.syride.com/fr/pilotes/MathiasDo/2458249
https://www.syride.com/fr/pilotes/nico_guehl/2458211
https://www.syride.com/fr/pilotes/Joh/2458853
https://www.syride.com/fr/pilotes/Santrave/2458081
https://www.syride.com/fr/pilotes/wis108/2458089
https://www.syride.com/fr/pilotes/Seb-b/2457493
https://www.syride.com/fr/pilotes/Rhum/2451558
https://www.syride.com/fr/pilotes/prabert/2450454
https://www.syride.com/fr/pilotes/Flyin_Matmute/2450480
https://www.syride.com/fr/pilotes/AirDhuile/2442102
https://www.syride.com/fr/pilotes/Carcarias/2442186
https://www.syride.com/fr/pilotes/Alain_73800/2442131
https://www.syride.com/fr/pilotes/Brewal/2442110
https://www.syride.com/fr/pilotes/benjaminff/2442077
https://www.syride.com/fr/pilotes/Mehdaye/2442532
https://www.syride.com/fr/pilotes/guillaumeR38/2442505
https://www.syride.com/fr/pilotes/Mermath38/2441983
https://www.syride.com/fr/pi

In [7]:
from selenium import webdriver
import requests
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
driver=webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

# Lien de la page web à scraper
url = "https://www.syride.com/fr/explorer/3764768529"
driver.get(url)

# Attendre que la page se charge complètement
driver.implicitly_wait(5)  # Attendre jusqu'à 10 secondes pour que les éléments se chargent

# Récupérer toutes les classes lineDiv
lineDivs = driver.find_elements(By.CLASS_NAME, "lineDiv")
vols=[]
# Pour chaque classe lineDiv, cliquer sur le bouton loupe et extraire le lien
for lineDiv in lineDivs:
    loupe_button = lineDiv.find_element(By.CSS_SELECTOR, "td[onclick^='showFlight'] img")
    driver.implicitly_wait(1)
    driver.execute_script("arguments[0].click();", loupe_button)
 
    vol_url = driver.current_url
    vols.append(vol_url)


for vol in vols:
    print(vol)
    driver.get(vol)
    driver.implicitly_wait(5) 
    # Si l'élément est dans un iframe, passez-y
    iframe = driver.find_element(By.TAG_NAME, "iframe")
    if iframe:
        driver.switch_to.frame(iframe)

    # Trouver l'élément de flèche sur la page actuelle
    arrow_button = driver.find_element(By.ID, "bandeauDroiteDivArrow")


# Cliquer sur le bouton
    arrow_button.click()
    # Extract the HTML content of the page
    page_source = driver.page_source
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    
    print(soup)
    # Find the <a> tag containing the ZIP link
    zip_link = soup.find('a', href=lambda href: href and 'downloadZIP.php' in href)
    
    # Extract the href attribute value if the ZIP link is found
    if zip_link:
        zip_url = zip_link['href']
        print("ZIP link:", zip_url)
    else:
        print("ZIP link not found on this page")
    


    
    filename = zip_url.split('=')[-1] + ".zip"  # Utilisation de la partie après le dernier '=' comme nom de fichier

        # Télécharger le fichier ZIP
    zip_url="https://www.syride.com/"+zip_url
    zip_response = requests.get(zip_url)

        # Vérifier si la requête a réussi (statut 200)
    if zip_response.status_code == 200:
            # Écrire le contenu de la réponse dans un fichier
        with open(filename, 'wb') as f:
            f.write(zip_response.content)

        print(f"Le fichier ZIP a été téléchargé avec succès sous le nom '{filename}'.")
    else:
        print(f"La requête de téléchargement du fichier ZIP à partir du lien '{zip_url}' a échoué.")
    break
# Fermer le navigateur
driver.quit()


https://www.syride.com/fr/pilotes/Stephgoepp/2459041
<html class="js csstransforms3d" style=""><head>
<link href="/css/reset.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/css/header.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/css/main.css?1887285705" media="all" rel="stylesheet" type="text/css"/>
<link href="/css/vol.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/css/component.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/css/owl.carousel.min.css" rel="stylesheet" type="text/css"/>
<link href="/css/owl.theme.default.css" rel="stylesheet" type="text/css"/>
<link href="/css/jcarousel.transitions.live.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/css/reaction.css?182496400" rel="stylesheet" type="text/css"/>
<script async="" crossorigin="anonymous" src="https://connect.facebook.net/fr_FR/sdk.js?hash=657b2a3fb7e235d8997e77ab1a0ee1dc"></script><script id="facebook-jssdk" src="//connect.facebook.n